# JAX Quickstart
https://jax.readthedocs.io/en/latest/notebooks/quickstart.html をなぞるだけ

## Multiplying Matrices

In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

In [2]:
key = random.PRNGKey(0)
x = random.normal(key, (10, ))
print(x)

[-0.3721109   0.26423115 -0.18252768 -0.7368197  -0.44030377 -0.1521442
 -0.67135346 -0.5908641   0.73168886  0.5673026 ]


In [4]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()

3.36 ms ± 17.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


JAX は numpy array をそのまま使用することもできるが，毎回GPUに移すので処理が遅くなるらしい

In [5]:
import numpy as np
x = np.random.normal(size=(size, size)).astype(np.float32)
%timeit jnp.dot(x, x.T).block_until_ready()

14.2 ms ± 498 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


`device_put(x)` (`to(device)`みたいなもの？)することで明示的にGPUに移すことができる

In [6]:
from jax import device_put

x = np.random.normal(size=(size, size)).astype(np.float32)
x = device_put(x)
%timeit jnp.dot(x, x.T).block_until_ready()

3.3 ms ± 43.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


JAX は単にGPUを利用したNumPyではなく，いくつか便利なコードをもっている．主には `jit()`: コードのスピードアップ, `grad()`: 微分, `vmap`: 自動ベクタライズやバッチ処理

## Using `jit()` to speed up functions
`@jit` デコレータによりまとまった処理をXLAを用いていコンパイルすることが可能

In [7]:
def selu(x, alpha=1.67, lmbda=1.05):
    return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

431 µs ± 6.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

34.8 µs ± 2.96 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Taking derivatives with `grad()`
`grad()`によって自動微分が可能

In [10]:
def sum_logistic(x):
    return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

x_small = jnp.arange(3.)
derivative_fn = grad(sum_logistic)
print(derivative_fn(x_small))

[0.25       0.19661194 0.10499357]


In [12]:
def first_finite_defferences(f, x):
    eps = 1e-3
    return jnp.array([(f(x + eps * v) - f(x - eps * v)) / (2 * eps)
                      for v in jnp.eye(len(x))])

print(first_finite_defferences(sum_logistic, x_small))

[0.24998187 0.1965761  0.10502338]


In [13]:
print(grad(jit(grad(jit(grad(sum_logistic)))))(1.0))

-0.0353256


## Auto-vectorization with `vmap()`

In [15]:
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))

def apply_matrix(v):
    return jnp.dot(mat, v)

各バッチに対してループで処理すると時間がかかる

In [17]:
def naively_batched_apply_matrix(v_batched):
    return jnp.stack([apply_matrix(v) for v in v_batched])

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

Naively batched
1.51 ms ± 7.82 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


`@jit`により手動でループ処理を書き処理をベルトル化することで高速

In [20]:
@jit
def batched_apply_matrix(v_batched):
    return jnp.dot(v_batched, mat.T)

print('Manually batched')
%timeit batched_apply_matrix(batched_x).block_until_ready()

Manually batched
26.3 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


`vmap()`を使用することで，バッチ処理をサポートしていない処理に対して自動でバッチ処理サポートを対応させることが可能

In [21]:
@jit
def vmap_batched_apply_matrix(v_batched):
    return vmap(apply_matrix)(v_batched)

print('Auto-vectorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

Auto-vectorized with vmap
26.1 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
